## Trabajo final

Obtencion de los datos de criptomonedas.

Usamos el exchanges de cryptos HitBTC, utilizamos la **API** de **demo.hitbtc.com**


Acerca de esta API (documentacion):
https://api.demo.hitbtc.com/#about-hitbtc-api

Vamos a trabajar con el mercado spot (contado) y este exchange opera con casi 400 crytos diferentes.

In [1]:
import requests
import pandas as pd

Listado de monedas

Definimos una funcion para obterner el listado de monedas con las cuales se puede operar.

In [2]:
def listaCryptos():
    url= 'https://api.hitbtc.com/api/2/public/currency'
    r= requests.get(url)
    js= r.json()
    df= pd.DataFrame(js).set_index('id')
    return df

In [3]:
cryptos= listaCryptos()
cryptos.loc[:,['fullName']]

,fullName
id,
DDF,DDF
ZRX,0x Protocol
ACO,ACO
ACT,Achain
ADX,AdEx
...,...
SFP,SafePal
OM,MANTRA DAO
PERL,Perlin


In [4]:
cryptos.reset_index(inplace=True)

In [5]:
pd.set_option('display.max_rows',100)

In [6]:
mask= (cryptos.id == 'BTC') |(cryptos.id == 'ETH')| (cryptos.id == 'XRP') | (cryptos.id == 'ADA') 
cryptos[mask]

,id,fullName,crypto,payinEnabled,payinPaymentId,payinConfirmations,payoutEnabled,payoutIsPaymentId,transferEnabled,delisted,payoutFee,precisionPayout,precisionTransfer,lowProcessingTime,highProcessingTime,avgProcessingTime
36,BTC,Bitcoin,True,True,False,1,True,False,True,False,0.000900000000,8,8,37.354,2692.176,284.626744680851
58,ADA,Cardano,True,True,False,10,True,False,True,False,4.470000000000,6,6,7.197,7372.74,243.39714432989692
131,ETH,Ethereum,True,True,False,9,True,False,True,False,0.011600000000,18,12,1.443,28.17,9.210799999999999
269,XRP,Ripple,True,True,True,20,True,True,True,False,12.790000000000,6,6,2.305,2696.58,36.11042268041239


### Obtenemos los precios historicos para una criptomoneda

Los valores que acepta la API para los periodos:

<li>M1, 1 minuto
<li>M3, 3 minutos
<li>M5, 5 minutos
<li>M15, 15 minutos
<li>M30, 30 minutos
<li>H1, 1 hora
<li>H4, 4 horas
<li>D1, 1 dia
<li>D7, 7 dias
<li>1M, 1 mes

In [7]:
# funcion para traer las velas de la criptomoneda a traves de la API del exchange
# sort values= 'DESC' o 'ASC'
def getVelas(crypto, periodo, desde, hasta, limite=1000, sort='ASC'):
    url= "https://api.hitbtc.com/api/2/public/candles"
    p= {'limit':limite, 'symbols': crypto, 'from': desde, 'till': hasta,
        'period':periodo,'sort': sort}
    r= requests.get(url, params= p)
    js= r.json()[crypto]
    df= pd.DataFrame(js)
    df['crypto']= crypto
    return df

- EL proceso esta armado con extraccion diaria, las cryptos se eligen en la lista_cryptos y los periodos en lista_desde y lista_hasta

In [8]:
# bajar del exchange los datos de las criptomonedas seleccionadas
lista_cryptos= ['BTCUSD','ETHUSD','ADAUSD']
lista_desde = ['2013-01-01','2014-01-01','2015-01-01','2016-01-01','2017-01-01',
               '2018-01-01','2019-01-01','2020-01-01','2021-01-01']
lista_hasta = ['2013-12-31','2014-12-31','2015-12-31','2016-12-31','2017-12-31',
               '2018-12-31','2019-12-31','2020-12-31','2021-12-31']

# el desde es a partir de que fecha trae las velas
# el hasta es la fecha hasta la que se trae los valores hasta el valor limite que por
# default que es 1.000. Mil es le maximo de datos que trae esta demo API
i= 0
for c in lista_cryptos: 
    for ix in range(len(lista_desde)):
        if i== 0:
            datos= getVelas(c, 'D1', desde=lista_desde[ix], hasta=lista_hasta[ix])
            i =  i + 1
        else:
            datos= datos.append(getVelas(c, 'D1', desde=lista_desde[ix], hasta=lista_hasta[ix]),
                   ignore_index=True)
        

- Guardamos la extraccion en nuestro archivo de velas

In [9]:
datos_path= '../Datos/'
file= 'velas_cryptos'
extencion= '.csv'
datos.to_csv(datos_path+file+extencion, sep=',', index=False)